# Assignmnet 1 DBMS INFO-6210

## The dataset selected is: Common Airline Customer Service Utterances 
It is a sample dataset created by CrowdFlower at https://www.crowdflower.com/wp-content/uploads/2017/11/Utterance-Flights-f1197494.csv on various customer service scenarios.
                

## Importing all dependencies

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy import stats
import seaborn as sns
from datetime import datetime
import sqlite3

# Reading data using pandas library

In [3]:
flight_data = pd.read_csv('flight_data.csv')

In [4]:
flight_data.head() #Printing the data

,UNIT ID,CREATED AT,ID,STARTED AT,TAINTED,CHANNEL,CHANNEL_ID,TRUST,WORKER ID,COUNTRY,REGION,CITY,IP,RESPONSE 1,RESPONSE 2,RESPONSE 3,SCENARIO,COMPOSITE_KEY
0,1403461953,10-11-17 22:56,2909781477,10-11-17 22:46,False,clixsense,1,1,39347443,AUT,9,Vienna,80.110.82.64,"I have some health problems, is it possible to...","I'm stuck in heavy traffic, can I change my fl...","I forgot my passport, could you change my flig...",You need to change your flight,28.0
1,1403461975,10-11-17 22:56,2909781471,10-11-17 22:46,False,clixsense,1,1,39347443,AUT,9,Vienna,80.110.82.64,Could you please check the direct flights to m...,"I need a direct flight to my location, do you ...",Do you fly directly maybe to my location?,You need to check if there are any direct flig...,28.0
2,1403461968,10-11-17 22:32,2909750917,10-11-17 22:23,False,clixsense,1,1,39347443,AUT,9,Vienna,80.110.82.64,"I think my bag is to heavy, what is the maximu...",What is the maximum weight for checked bag?,"I have more space in my bag, what is the maxim...",You need to know the maximum weight for a chec...,28.0
3,1403461973,10-11-17 22:32,2909750911,10-11-17 22:23,False,clixsense,1,1,39347443,AUT,9,Vienna,80.110.82.64,"I am injured, could you get me a wheelchair?","I can't get out on my own, I would like a whee...",Is it possible to get a wheelchair?,You need to request a wheelchair when you land,28.0
4,1403461959,10-11-17 22:56,2909781476,10-11-17 22:46,False,clixsense,1,1,39347443,AUT,9,Vienna,80.110.82.64,Could you please tell me my frequent flier num...,What type of ID do you need to tell me my freq...,"I have lost my frequent flier number, could yo...",You need your frequent flier number,28.0


In [5]:
flight_data.isnull().sum() #Checking all the null values

UNIT ID            0
CREATED AT         0
ID                 0
STARTED AT         0
TAINTED            0
CHANNEL            0
CHANNEL_ID         0
TRUST              0
WORKER ID          0
COUNTRY            0
REGION           165
CITY             165
IP                 0
RESPONSE 1         0
RESPONSE 2         1
RESPONSE 3         0
SCENARIO           8
COMPOSITE_KEY    165
dtype: int64

## We now clean and reformat the data to fit the database schema

The data contains many NULL values and a few columns which are redundant and are not needed to be stored in our database. In the following clleaning comands we shall take care of them and fit our data to the SQL schema we have designed.


In [6]:
flight_data.drop(['CREATED AT', 'STARTED AT', 'TRUST', 'TAINTED'], axis=1, inplace = True) #Droping all the redundant attributes

In [7]:
flight_data = flight_data.dropna() #Removing all the NUll values

In [8]:
flight_data.isnull().sum() #Checking for any NULL values

UNIT ID          0
ID               0
CHANNEL          0
CHANNEL_ID       0
WORKER ID        0
COUNTRY          0
REGION           0
CITY             0
IP               0
RESPONSE 1       0
RESPONSE 2       0
RESPONSE 3       0
SCENARIO         0
COMPOSITE_KEY    0
dtype: int64

The data as we see has no null values and the redundant rows have been removed too, thus we can say that the data is complete, consistant and uniform. Thus, we can store this dataset into a database using SQLite3.

## Formating the strings to same cases

This makes the storage into the databases easier.

In [9]:
flight_data = flight_data.apply(lambda x: x.astype(str).str.lower())
flight_data.head()

,UNIT ID,ID,CHANNEL,CHANNEL_ID,WORKER ID,COUNTRY,REGION,CITY,IP,RESPONSE 1,RESPONSE 2,RESPONSE 3,SCENARIO,COMPOSITE_KEY
0,1403461953,2909781477,clixsense,1,39347443,aut,9,vienna,80.110.82.64,"i have some health problems, is it possible to...","i'm stuck in heavy traffic, can i change my fl...","i forgot my passport, could you change my flig...",you need to change your flight,28.0
1,1403461975,2909781471,clixsense,1,39347443,aut,9,vienna,80.110.82.64,could you please check the direct flights to m...,"i need a direct flight to my location, do you ...",do you fly directly maybe to my location?,you need to check if there are any direct flig...,28.0
2,1403461968,2909750917,clixsense,1,39347443,aut,9,vienna,80.110.82.64,"i think my bag is to heavy, what is the maximu...",what is the maximum weight for checked bag?,"i have more space in my bag, what is the maxim...",you need to know the maximum weight for a chec...,28.0
3,1403461973,2909750911,clixsense,1,39347443,aut,9,vienna,80.110.82.64,"i am injured, could you get me a wheelchair?","i can't get out on my own, i would like a whee...",is it possible to get a wheelchair?,you need to request a wheelchair when you land,28.0
4,1403461959,2909781476,clixsense,1,39347443,aut,9,vienna,80.110.82.64,could you please tell me my frequent flier num...,what type of id do you need to tell me my freq...,"i have lost my frequent flier number, could yo...",you need your frequent flier number,28.0


## Creating a connection to the database



In [10]:
conn = sqlite3.connect("FlightSurvey.db") #connects to a database named "FlightSurvey"

In [11]:
c = conn.cursor()

## Splitting data into various tables (Normalizing Data)



# Table1: Customer response (Main Table)



In [12]:
cust_responses = pd.DataFrame()
cust_responses[['RESPONSE_ID','RESPONSE_1','RESPONSE_2','RESPONSE_3','UNIT_ID','WORKER_ID','CHANNEL_ID','REGION_ID']] = flight_data[['ID', 'RESPONSE 1', 'RESPONSE 2', 'RESPONSE 3', 'UNIT ID', 'WORKER ID','CHANNEL_ID','COMPOSITE_KEY']]
cust_responses.head()

,RESPONSE_ID,RESPONSE_1,RESPONSE_2,RESPONSE_3,UNIT_ID,WORKER_ID,CHANNEL_ID,REGION_ID
0,2909781477,"i have some health problems, is it possible to...","i'm stuck in heavy traffic, can i change my fl...","i forgot my passport, could you change my flig...",1403461953,39347443,1,28.0
1,2909781471,could you please check the direct flights to m...,"i need a direct flight to my location, do you ...",do you fly directly maybe to my location?,1403461975,39347443,1,28.0
2,2909750917,"i think my bag is to heavy, what is the maximu...",what is the maximum weight for checked bag?,"i have more space in my bag, what is the maxim...",1403461968,39347443,1,28.0
3,2909750911,"i am injured, could you get me a wheelchair?","i can't get out on my own, i would like a whee...",is it possible to get a wheelchair?,1403461973,39347443,1,28.0
4,2909781476,could you please tell me my frequent flier num...,what type of id do you need to tell me my freq...,"i have lost my frequent flier number, could yo...",1403461959,39347443,1,28.0


# Table2: Scenarios (Dependency)



In [13]:
scenarios = pd.DataFrame()
scenarios[['SCENARIO_ID','SCENARIO']] = flight_data[['UNIT ID', 'SCENARIO']]
scenarios = scenarios.drop_duplicates() #dropping duplicates
scenarios.head()

,SCENARIO_ID,SCENARIO
0,1403461953,you need to change your flight
1,1403461975,you need to check if there are any direct flig...
2,1403461968,you need to know the maximum weight for a chec...
3,1403461973,you need to request a wheelchair when you land
4,1403461959,you need your frequent flier number


# Table3 : Workers (Dependency)




In [14]:
workers = pd.DataFrame()
workers[['WORKER_ID','IP']] = flight_data[['WORKER ID','IP']]
workers = workers.drop_duplicates() #eliminating duplicates
workers.head()

,WORKER_ID,IP
0,39347443,80.110.82.64
10,38372558,200.163.232.161
12,43125727,179.180.13.10
13,43847563,138.0.21.97
14,43853403,179.158.154.94


#  Table4: Channel (Dependency)

In [15]:
channel = pd.DataFrame()
channel[['CHANNEL_ID','CHANNEL']] = flight_data[['CHANNEL_ID','CHANNEL']]
channel = channel.drop_duplicates()
channel

,CHANNEL_ID,CHANNEL
0,1,clixsense
10,2,neodev
12,3,elite
50,4,prodege
691,5,bitcoinget
776,6,gifthunterclub


# Table5: Regions (Dependency)

In [16]:
region = pd.DataFrame()
region[['REGION_ID','COUNTRY','CITY','REGION']] = flight_data[['COMPOSITE_KEY','COUNTRY','CITY','REGION']]
region = region.drop_duplicates()
region.head()

,REGION_ID,COUNTRY,CITY,REGION
0,28.0,aut,vienna,9
10,56.0,bra,goiania,29
12,57.0,bra,recife,30
13,20.0,bra,juazeiro,5
14,50.0,bra,rio de janeiro,21


# Creating the SQL tables

Now that we have normalized our flat file completely, we can store the tables as SQL tables in the database

## Table1 to SQL (Primary table with all the Primary and Foreign keys)

In [17]:
c.execute("""drop table cust_responses""")
c.execute("""CREATE TABLE cust_responses(
response_id INTEGER PRIMARY KEY,
response_1 CHAR(150),
response_2 CHAR(150),
response_3 CHAR(150),
unit_id INTEGER FOREGION KEY,
worker_id INTEGER FOREGION KEY,
channel_id INTEGER FOREGION KEY,
region_id INTEGER FOREGION KEY);""") #Creating the new table and defining the schema

## We start to define the schema for the various tables

In [18]:
cust_responses.to_sql("cust_responses", conn, if_exists = "append", index = False) #Storing data frame to SQL database

In [19]:
pd.read_sql_query("""select * from cust_responses;""", conn).head() #Querying the database for data

,response_id,response_1,response_2,response_3,unit_id,worker_id,channel_id,region_id
0,2909628960,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461960,43650485,2,31
1,2909628961,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461972,43650485,2,31
2,2909628963,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461979,43650485,2,31
3,2909628964,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461970,43650485,2,31
4,2909628965,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461964,43650485,2,31


# Table2 to SQL (Dependent table)

In [20]:
c.execute("""drop table scenarios""")
c.execute("""CREATE TABLE scenarios(
scenario_id INTEGER PRIMARY KEY,
scenario CHAR(150));""") 

In [21]:
scenarios.to_sql("scenarios", conn, if_exists = "append", index = False)
pd.read_sql_query("""select * from scenarios;""", conn).head()

,scenario_id,scenario
0,1403461952,your bag was lost
1,1403461953,you need to change your flight
2,1403461954,you want to know if it costs money to bring a ...
3,1403461955,you had a bad flight and want some money back
4,1403461956,you want to cancel your flight


## Table3 to SQL (Dependent table)

In [22]:
c.execute("""drop table channel""")
c.execute("""CREATE TABLE channel(
channel_id INTEGER PRIMARY KEY,
channel CHAR(20));""")

In [23]:
channel.to_sql("channel", conn, if_exists = "append", index = False)
pd.read_sql_query("""select * from channel;""", conn)

,channel_id,channel
0,1,clixsense
1,2,neodev
2,3,elite
3,4,prodege
4,5,bitcoinget
5,6,gifthunterclub


## Table4 to SQL (Dependent table)

In [24]:
c.execute("""drop table region""")
c.execute("""CREATE TABLE region(
region_id INTEGER PRIMARY KEY,
country CHAR(20),
city CHAR(20),
region CHAR(5));""")

In [25]:
region.to_sql("region", conn, if_exists = "append", index = False)
pd.read_sql_query("""select * from region;""", conn).head()

,region_id,country,city,region
0,11,mne,budva,0
1,12,srb,belgrade,0
2,13,cri,grecia,1
3,14,gha,accra,1
4,15,col,medellín,2


## Table5 to SQL (Dependent table)

In [26]:
c.execute("""drop table workers""")
c.execute("""CREATE TABLE workers(
worker_id INTEGER PRIMARY KEY,
ip CHAR(20));""")

In [27]:
workers.to_sql("workers", conn, if_exists = "append", index = False)
pd.read_sql_query("""select * from workers;""", conn).head()

,worker_id,ip
0,5551648,173.183.76.192
1,10422922,39.48.57.27
2,12291277,78.62.238.184
3,21764533,78.2.113.83
4,32512686,200.93.70.139


# Usecases

## Executing the SELECT - FROM - WHERE query

In [28]:
pd.read_sql_query("""SELECT response_1, scenario FROM cust_responses AS c, scenarios AS s WHERE c.unit_id = s.scenario_id;""",conn).head()

,response_1,scenario
0,are taxis available to book right now?,you want to know how much it’s going to cost t...
1,are taxis available to book right now?,you’re worried about the weather and its impli...
2,are taxis available to book right now?,you want to add travel insurance to a purchase...
3,are taxis available to book right now?,you are threatening to never to use this airli...
4,are taxis available to book right now?,you need to change your flight


## Executing the SELECT - FROM - WHERE-LIMIT query

In [29]:
pd.read_sql_query("""SELECT response_1, response_2, scenario FROM cust_responses AS c, scenarios AS s WHERE c.unit_id = s.scenario_id LIMIT 10;""",conn)

,response_1,response_2,scenario
0,are taxis available to book right now?,are taxis available to book right now?,you want to know how much it’s going to cost t...
1,are taxis available to book right now?,are taxis available to book right now?,you’re worried about the weather and its impli...
2,are taxis available to book right now?,are taxis available to book right now?,you want to add travel insurance to a purchase...
3,are taxis available to book right now?,are taxis available to book right now?,you are threatening to never to use this airli...
4,are taxis available to book right now?,are taxis available to book right now?,you need to change your flight
5,can i change the date ?,can i change the date ?,you booked a flight for the wrong day and need...
6,there is a travel advisory in the place you’re...,there is a travel advisory in the place you’re...,you need to see if there is a travel advisory ...
7,can you help me get a new flight?,can you help me get a new flight?,your flight was cancelled and you need to get ...
8,can you help me get a new flight?,can you help me get a new flight?,you want to cancel your flight
9,can you help me get a new flight?,can you help me get a new flight?,you want to check if your flight is on time


## Executing a simple JOIN using SELECT - FROM - WHERE query

The number of rows returned is 1025 which is the same as the number of rows in the orignal flat table. This signifies that the normalization is perfect.

In [30]:
pd.read_sql_query("""SELECT * FROM cust_responses AS c, scenarios AS s, workers AS w, channel AS ch, region AS r WHERE c.unit_id = s.scenario_id AND c.worker_id = w.worker_id AND c.channel_id = ch.channel_id AND c.region_id = r.region_id;""",conn)

,response_id,response_1,response_2,response_3,unit_id,worker_id,channel_id,region_id,scenario_id,scenario,worker_id,ip,channel_id,channel,region_id,country,city,region
0,2909628960,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461960,43650485,2,31,1403461960,you want to know how much it’s going to cost t...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
1,2909628961,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461972,43650485,2,31,1403461972,you’re worried about the weather and its impli...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
2,2909628963,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461979,43650485,2,31,1403461979,you want to add travel insurance to a purchase...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
3,2909628964,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461970,43650485,2,31,1403461970,you are threatening to never to use this airli...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
4,2909628965,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461964,43650485,2,31,1403461964,you need to change your flight,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
5,2909631961,can i change the date ?,can i change the date ?,can i change the date ?,1403461974,43650485,2,31,1403461974,you booked a flight for the wrong day and need...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
6,2909631963,there is a travel advisory in the place you’re...,there is a travel advisory in the place you’re...,there is a travel advisory in the place you’re...,1403461976,43650485,2,31,1403461976,you need to see if there is a travel advisory ...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
7,2909631966,can you help me get a new flight?,can you help me get a new flight?,can you help me get a new flight?,1403461962,43650485,2,31,1403461962,your flight was cancelled and you need to get ...,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
8,2909631968,can you help me get a new flight?,can you help me get a new flight?,can you help me get a new flight?,1403461956,43650485,2,31,1403461956,you want to cancel your flight,43650485,156.211.251.26,2,neodev,31,egy,cairo,11
9,2909631969,can you help me get a new flight?,can you help me get a new flight?,can you help me get a new flight?,1403461957,43650485,2,31,1403461957,you want to check if your flight is on time,43650485,156.211.251.26,2,neodev,31,egy,cairo,11


## Executing a query using INNER JOIN (two tables)

In [31]:
pd.read_sql_query("""SELECT response_1, channel FROM cust_responses AS c INNER JOIN channel AS ch ON c.channel_id = ch.channel_id;""", conn).head()

,response_1,channel
0,are taxis available to book right now?,neodev
1,are taxis available to book right now?,neodev
2,are taxis available to book right now?,neodev
3,are taxis available to book right now?,neodev
4,are taxis available to book right now?,neodev


## Executing a query using LEFT OUTER JOIN (two tables)

In [32]:
pd.read_sql_query("""SELECT response_1, channel FROM cust_responses AS c LEFT JOIN channel AS ch ON c.channel_id = ch.channel_id;""", conn).head()

,response_1,channel
0,are taxis available to book right now?,neodev
1,are taxis available to book right now?,neodev
2,are taxis available to book right now?,neodev
3,are taxis available to book right now?,neodev
4,are taxis available to book right now?,neodev


## Executing the query using LEFT OUTER JOIN (three tables)

In [33]:
pd.read_sql_query("""SELECT response_1, channel, ip FROM cust_responses AS c LEFT JOIN channel AS ch, workers AS w ON c.channel_id = ch.channel_id AND c.worker_id = w. worker_id;""", conn).head()

,response_1,channel,ip
0,are taxis available to book right now?,neodev,156.211.251.26
1,are taxis available to book right now?,neodev,156.211.251.26
2,are taxis available to book right now?,neodev,156.211.251.26
3,are taxis available to book right now?,neodev,156.211.251.26
4,are taxis available to book right now?,neodev,156.211.251.26


## Executing query using INNER JOIN (three tables)

In [34]:
pd.read_sql_query("""SELECT response_1, channel, country FROM cust_responses AS c INNER JOIN channel AS ch, region AS r ON c.channel_id = ch.channel_id AND c.region_id = r.region_id ;""", conn).head()

,response_1,channel,country
0,are taxis available to book right now?,neodev,egy
1,are taxis available to book right now?,neodev,egy
2,are taxis available to book right now?,neodev,egy
3,are taxis available to book right now?,neodev,egy
4,are taxis available to book right now?,neodev,egy


## ~**No support for  RIGHT OUTER and FULL JOINS in SQLite in python*~

## Executing INNER JOIN on all the tables

In [35]:
pd.read_sql_query("""SELECT * FROM cust_responses AS c INNER JOIN channel AS ch, region AS r, scenarios AS s, workers AS w ON c.worker_id = w.worker_id AND c.unit_id = s.scenario_id AND c.channel_id = ch. channel_id AND c.region_id = r.region_id; """,conn).head()

,response_id,response_1,response_2,response_3,unit_id,worker_id,channel_id,region_id,channel_id,channel,region_id,country,city,region,scenario_id,scenario,worker_id,ip
0,2909628960,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461960,43650485,2,31,2,neodev,31,egy,cairo,11,1403461960,you want to know how much it’s going to cost t...,43650485,156.211.251.26
1,2909628961,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461972,43650485,2,31,2,neodev,31,egy,cairo,11,1403461972,you’re worried about the weather and its impli...,43650485,156.211.251.26
2,2909628963,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461979,43650485,2,31,2,neodev,31,egy,cairo,11,1403461979,you want to add travel insurance to a purchase...,43650485,156.211.251.26
3,2909628964,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461970,43650485,2,31,2,neodev,31,egy,cairo,11,1403461970,you are threatening to never to use this airli...,43650485,156.211.251.26
4,2909628965,are taxis available to book right now?,are taxis available to book right now?,are taxis available to book right now?,1403461964,43650485,2,31,2,neodev,31,egy,cairo,11,1403461964,you need to change your flight,43650485,156.211.251.26


In [36]:
conn.commit()
conn.close()

I have shown various usecases in the exampples above. The schema of the given database is such that there is only one parent table which has 4 connected child tables. The parent table has the primary keys of the child table as foreign keys.
